In [4]:
import pandas as pd
import polars as pl
import zipfile
import os
import pypyodbc as odbc
from dotenv import load_dotenv
from sqlalchemy import create_engine
import urllib

In [5]:
load_dotenv()

True

In [ ]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = os.environ.get('server_name')
DATABASE_NAME = 'ReceitaFederal'

connection_string = f"""
DRIVER={{{DRIVER_NAME}}};
SERVER={SERVER_NAME};
DATABASE={DATABASE_NAME};
Trusted_Connection=yes;
"""

conn = odbc.connect(connectString=connection_string)
print(conn)

In [ ]:
DRIVER = "ODBC Driver 17 for SQL Server"
SERVER = os.environ.get("server_name")
DATABASE = "ReceitaFederal"
USERNAME = os.environ.get("user")
PASSWORD = os.environ.get("password")

params = urllib.parse.quote_plus(
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD};"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [9]:
columnsEstabelecimentos = [
    ("CNPJ BÁSICO", "BIGINT", "NULL"),
    ("CNPJ ORDEM", "BIGINT", "NULL"),
    ("CNPJ DV", "BIGINT", "NULL"),
    ("IDENTIFICADOR MATRIZ/FILIAL", "BIGINT", "NULL"),
    ("NOME FANTASIA", "VARCHAR(MAX)", "NULL"),
    ("SITUAÇÃO CADASTRAL", "BIGINT", "NULL"),
    ("DATA SITUAÇÃO CADASTRAL", "BIGINT", "NULL"),
    ("MOTIVO SITUAÇÃO CADASTRAL", "BIGINT", "NULL"),
    ("NOME DA CIDADE NO EXTERIOR", "VARCHAR(MAX)", "NULL"),
    ("PAIS", "VARCHAR(MAX)", "NULL"),
    ("DATA DE INÍCIO ATIVIDADE", "BIGINT", "NULL"),
    ("CNAE FISCAL PRINCIPAL", "BIGINT", "NULL"),
    ("CNAE FISCAL SECUNDÁRIA", "VARCHAR(MAX)", "NULL"),
    ("TIPO DE LOGRADOURO", "VARCHAR(MAX)", "NULL"),
    ("LOGRADOURO", "VARCHAR(MAX)", "NULL"),
    ("NÚMERO", "VARCHAR(MAX)", "NULL"),
    ("COMPLEMENTO", "VARCHAR(MAX)", "NULL"),
    ("BAIRRO", "VARCHAR(MAX)", "NULL"),
    ("CEP", "VARCHAR(MAX)", "NULL"),
    ("UF", "VARCHAR(MAX)", "NULL"),
    ("MUNICÍPIO", "BIGINT", "NULL"),
    ("DDD 1", "VARCHAR(MAX)", "NULL"),
    ("TELEFONE 1", "VARCHAR(MAX)", "NULL"),
    ("DDD 2", "VARCHAR(MAX)", "NULL"),
    ("TELEFONE 2", "VARCHAR(MAX)", "NULL"),
    ("DDD DO FAX", "VARCHAR(MAX)", "NULL"),
    ("FAX", "VARCHAR(MAX)", "NULL"),
    ("CORREIO ELETRÔNICO", "VARCHAR(MAX)", "NULL"),
    ("SITUAÇÃO ESPECIAL", "VARCHAR(MAX)", "NULL"),
    ("DATA DA SITUAÇÃO ESPECIAL", "VARCHAR(MAX)", "NULL")
]


columnsEmpresas = [
    ("CNPJ BÁSICO", "BIGINT", "NULL"),
    ("RAZÃO SOCIAL / NOME EMPRESARIAL", "VARCHAR(MAX)", "NULL"),
    ("NATUREZA JURÍDICA", "BIGINT", "NULL"),
    ("QUALIFICAÇÃO DO RESPONSÁVEL", "BIGINT", "NULL"),
    ("CAPITAL SOCIAL DA EMPRESA", "VARCHAR(MAX)", "NULL"),
    ("PORTE DA EMPRESA", "BIGINT", "NULL"),
    ("ENTE FEDERATIVO RESPONSÁVEL", "VARCHAR(MAX)", "NULL")
]

columnsSimples= [
    ("CNPJ BÁSICO", "INT", "NULL"),
    ("OPÇÃO PELO SIMPLES", "NVARCHAR(255)", "NULL"),
    ("DATA DE OPÇÃO PELO SIMPLES", "INT", "NULL"),
    ("DATA DE EXCLUSÃO DO SIMPLES", "INT", "NULL"),
    ("OPÇÃO PELO MEI", "NVARCHAR(255)", "NULL"),
    ("DATA DE OPÇÃO PELO MEI", "INT", "NULL"),
    ("DATA DE EXCLUSÃO DO MEI", "INT", "NULL")
]

In [ ]:
pathEmpresas = "DadosRF/Empresas"
pathEstabelecimentos = "DadosRF/Estabelecimentos"
pathSimples = "DadosRF/Simples"

zipFilesEmpresas = os.listdir(pathEmpresas)
zipFilesEstabelecimentos = os.listdir(pathEstabelecimentos)
zipFilesSimples = os.listdir(pathSimples)

In [11]:
def FixPath(files, basePath):
    filesPath = []
    for file in files:
        filesPath.append(os.path.join(basePath, file)) 
        
    return filesPath 

In [ ]:
filesEmpresas = FixPath(zipFilesEmpresas, pathEmpresas)
fileEstabelecimentos = FixPath(zipFilesEstabelecimentos, pathEstabelecimentos)
fileSimples = FixPath(zipFilesSimples, pathSimples)

In [51]:
for file in filesEmpresas:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("TratamentoInicial/EmpresasUnzip")

for file in fileEstabelecimentos:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("TratamentoInicial/EstabelecimentosUnzip")

for file in fileSimples:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("TratamentoInicial/SimplesUnzip")


In [13]:
filesEmpresasUnzip = os.listdir("TratamentoInicial/EmpresasUnzip")
fileEstabelecimentosUnzip = os.listdir("TratamentoInicial/EstabelecimentosUnzip")
fileSimplesUnzip = os.listdir("TratamentoInicial/SimplesUnzip")

filesEmpresas = FixPath(filesEmpresasUnzip, "TratamentoInicial/EmpresasUnzip")
fileEstabelecimentos = FixPath(fileEstabelecimentosUnzip, "TratamentoInicial/EstabelecimentosUnzip")
fileSimples = FixPath(fileSimplesUnzip, "TratamentoInicial/SimplesUnzip")


In [ ]:
def infer_sql_type(series: pd.Series) -> str:
    
    if pd.api.types.is_integer_dtype(series):
        return "BIGINT"
    elif pd.api.types.is_float_dtype(series):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(series):
        return "BIGINT"
    else:
        max_len = series.dropna().astype(str).map(len).max()
        length = max(600, max_len)  
        return f"NVARCHAR({length})"

In [ ]:
def bulk_insert(data_file, table, columns_info):
    
    columns_sql = ",\n    ".join([f"[{col[0]}] {col[1]} {col[2]}" for col in columns_info])
    
    sql = f"""
            IF OBJECT_ID('{table}', 'U') IS NULL
            BEGIN
                CREATE TABLE {table} (
                    {columns_sql}
                );
            END;

            BULK INSERT {table}
            FROM '{data_file}'
            WITH (
                FIELDTERMINATOR = ';',
                ROWTERMINATOR = '\\n',
                FIRSTROW = 2,
                CODEPAGE = '1252',
                TABLOCK
            );
            """.strip()
                
    return sql


In [16]:
def insertFiles(data_files, table, connection, column_names):

    cursor = connection.cursor()
    try:
        print("Begin insert operation")
        with cursor:
            print("Cursor avaliable")
            for data_file in data_files:
                print("checking", os.path.abspath(data_file))
                print("Inserting", os.path.abspath(data_file))
                cursor.execute(bulk_insert(os.path.abspath(data_file), table, column_names))
                print(data_file, "inserted")
            cursor.commit()
    except Exception as err:
        print(err)
        connection.rollback()
        print("Rollback has been made")

In [ ]:
insertFiles(filesEmpresas, "Empresas", conn, columnsEmpresas)

insertFiles(fileEstabelecimentos, "Estabelecimentos", conn, columnsEstabelecimentos)

insertFiles(fileSimples, "Simples", conn, columnsSimples)